**Table of contents**<a id='toc0_'></a>    
- [Title:](#toc1_1_1_)    
    - [Authors:](#toc1_1_2_)    
    - [Abstract:](#toc1_1_3_)    
    - [References](#toc1_1_4_)    
    - [Assumption](#toc1_1_5_)    
    - [Mathematical basis](#toc1_1_6_)    
    - [Numerical method](#toc1_1_7_)    
        - [Step 1) Simulate $X_u|X_t$ using the noncentral $\chi^2$ distribution](#toc1_1_7_1_1_)    
        - [Step 2) $\int_t^u \frac{ds}{X_s}$ Given $X_u, X_t$](#toc1_1_7_1_2_)    
        - [Step 3) Exact method: Simulate $V_T$ with CDF by Laplace transform](#toc1_1_7_1_3_)    
        - [Step 3) Almost exact method: Simulate $V_T$ with assumed distribution](#toc1_1_7_1_4_)    
        - [Finally get the option price](#toc1_1_7_1_5_)    
  - [Case I: Eq. (4.2) in Baldeaux (2012)](#toc1_2_)    
  - [Case II: Set 2 in Kouarfate et al. (2021)](#toc1_3_)    
  - [Case III:in Kouarfate et al. (2021)](#toc1_4_)    
  - [Case IV:](#toc1_5_)    
  - [Pricing with Time Discreteization using Euler/Milstein scheme, Exact Stepping, Almost Exact Stepping](#toc1_6_)    
  - [Pricing with Exact Simulation](#toc1_7_)    
  - [Pricing with IG approximation (Almost Exact Simulation)](#toc1_8_)    
- [Pricing with FFT method](#toc2_)    
- [Pricing with approximation IV](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_1_'></a>[Title:](#toc0_)
__Exact simulation and Almost exact simulation of the 3/2 model__

### <a id='toc1_1_2_'></a>[Authors:](#toc0_)
* Jan Baldeaux
* Choi J, Kwok YK
* Medvedev, A., Scaillet, O.

### <a id='toc1_1_3_'></a>[Abstract:](#toc0_)
* Baldeaux J (2012) Exact simulation of the 3/2 model. Int J Theor Appl Finan 15:1250032. https://doi.org/10.1142/S021902491250032X

This paper discusses the exact simulation of the stock price process underlying the 3/2 model. Using a result derived by Craddock and Lennox using Lie Symmetry Analysis, we adapt the Broadie-Kaya algorithm for the simulation of affine processes to the 3/2 model. We also discuss variance reduction techniques and find that conditional Monte Carlo techniques combined with quasi-Monte Carlo point sets result in significant variance reductions.

* Choi J, Kwok YK (2024) Simulation schemes for the Heston model with Poisson conditioning. European Journal of Operational Research 314(1):363–376. https://doi.org/10.1016/j.ejor.2023.10.048

Exact simulation schemes under the Heston stochastic volatility model (e.g., Broadie–Kaya and Glasserman–Kim) suffer from computationally expensive modified Bessel function evaluations. We propose a new exact simulation scheme without the modified Bessel function, based on the observation that the conditional integrated variance can be simplified when conditioned by the Poisson variate used for simulating the terminal variance. Our approach also enhances the low-bias and time discretization schemes, which are suitable for pricing derivatives with frequent monitoring. Extensive numerical tests reveal the good performance of the new simulation schemes in terms of accuracy, efficiency, and reliability when compared with existing methods.

* Medvedev, A., & Scaillet, O. (2007). Approximation and calibration of short-term implied volatilities under jump-diffusion stochastic volatility. The Review of Financial Studies, 20(2), 427-459.https://doi.org/10.1093/rfs/hhl013

We derive an asymptotic expansion formula for option implied volatility under a two factor jump-diffusion stochastic volatility model when time-to-maturity is small. We further propose a simple calibration procedure of an arbitrary parametric model to short-term near-the-money implied volatilities. An important advantage of our approximation is that it is free of the unobserved spot volatility. Therefore, the model can be calibrated on option data pooled across different calendar dates to extract information from the dynamics of the implied volatility smile. An example of calibration to a sample of S&P 500 option prices is provided. 


### <a id='toc1_1_4_'></a>[References](#toc0_)
* Baldeaux J (2012) Exact simulation of the 3/2 model. Int J Theor Appl Finan 15:1250032.
* Kouarfate IR, Kouritzin MA, MacKay A (2021) Explicit Solution Simulation Method for the 3/2 Model. In: Hernández‐Hernández D, Leonardi F, Mena RH, Pardo Millán JC (eds) Advances in Probability and Mathematical Statistics. Springer International Publishing, Cham, pp 123–145
* M. Jeanblanc, M. Yor and M. Chesney, Mathematical Methods for Financial Markets (Springer Finance, Springer, 2009).
* Choi J, Kwok YK (2024) Simulation schemes for the Heston model with Poisson conditioning. European Journal of Operational Research 314(1):363–376.
* Medvedev, A., & Scaillet, O. (2007). Approximation and calibration of short-term implied volatilities under jump-diffusion stochastic volatility. The Review of Financial Studies, 20(2), 427-459.

### <a id='toc1_1_5_'></a>[Assumption](#toc0_)

According to Baldeaux(2013), The dynamics of the stock price under the 3/2 model under the risk-neutral measure are given by

$$
 \frac{dS_t}{S_t} = rdt + \sqrt{V_t}\rho dW_t^1 + \sqrt{V_t}\sqrt{1-\rho^2}dW_t^2 \tag{1}
$$

$$
 \frac{dV_t}{V_t} = \kappa (\theta - V_t)dt + \epsilon \sqrt{V_t}dW_t^1
$$

which is equivalent to

$$
 dV_t = \kappa V_t (\theta - V_t)dt + \epsilon V_t^{3/2}dW_t^1 \tag{2}
$$

where $W_t^1$ and $W_t^2$ are independent Brownian motions. Regarding the parameters, $r$ represents the constant interest rate, $\rho$ the instantaneous correlation between the return on the stock and the variance process and $\epsilon$ governs the volatility of volatility.The speed of mean reversion is given by $\kappa V_t$ and $\theta$ denotes the long-run mean of the variance process.

### <a id='toc1_1_6_'></a>[Mathematical basis](#toc0_)

Defining $X_t = \frac{1}{V_t}$, we obtain

$$
dX_t = (\kappa + \epsilon^2 - \kappa\theta X_t)dt - \epsilon \sqrt{X_t}dW_t^1 \tag{3}
$$
Hence, using the process $X_t$, we obtain the following dynamics for the stock price, where $u > t$

$$
S_u = S_t \exp\lbrace r(u-t) - 1/2 \int_t^u(X_s)^{-1}ds + \rho \int_t^u({\sqrt{X_s})^{-1}dW_s^1}\rbrace \exp \lbrace \sqrt{1-\rho^2} \int_t^u(\sqrt{X_s})^{-1} dW_s^2\rbrace \tag{4}
$$
From Baldeaux(2013), study $\log(X_t)$

$$
\int_t^u({\sqrt{X_s})^{-1}dW_s^1} = \frac{1}{\epsilon} (log(\frac{X_t}{X_u}) + (k + \frac{\epsilon^2}{2})\int_{t}^{u}\frac{ds}{X_s}-k\theta(u-t)) \tag{5}
$$

Therefore, the only thing we need to know is the distribution of $X_t$ and $\int_{t}^{u}\frac{ds}{X_s}$ conditional on $X_t$

### <a id='toc1_1_7_'></a>[Numerical method](#toc0_)
Using Broadie-Kaya algorithm, we specify the simulation as 3 steps

##### <a id='toc1_1_7_1_1_'></a>[Step 1) Simulate $X_u|X_t$ using the noncentral $\chi^2$ distribution](#toc0_)
$X_u$ is distributed as a noncentral $ \chi^2 $ distribution

$$
\frac{X_u{\rm exp}\lbrace \kappa \theta (u-t) \rbrace}{c(u-t)} \sim \chi^2(\delta, \alpha) \tag{6}
$$

where
$$
\delta = \frac{4(\kappa + \epsilon^2)}{\epsilon^2}, \quad \alpha = \frac{X_t}{c(u-t)}, \quad c(t) = \frac{\epsilon^2({\rm exp}\lbrace \kappa\theta u \rbrace - 1)}{4\kappa\theta}
$$

##### <a id='toc1_1_7_1_2_'></a>[Step 2) $\int_t^u \frac{ds}{X_s}$ Given $X_u, X_t$](#toc0_)
We first derive the characteristic function of $\int_u^t \frac{ds}{X_s}$, which is provided in Baldeaux(2013)

$$
E\left({\rm exp}\left\lbrace -a^* \int_0^t \frac{ds}{X_s} \ \bigg| \ X_t \right\rbrace \right) = \frac{I_{\sqrt{\nu^2+8a/\epsilon^2}}\left(-\frac{j\sqrt{X_tX_u}}{{\rm sinh}\left(j\Delta\right)}\right)}{I_{\nu}\left(-\frac{j\sqrt{X_tX_u}}{{\rm sinh}\left(j\Delta\right)}\right)}\tag{7}
$$

where $j=-\frac{2\kappa\theta}{\epsilon^2}$, $\Delta=\frac{u\epsilon^2}{4}-\frac{t\epsilon^2}{4}$, $v=\frac{n}{2}-1$.

##### <a id='toc1_1_7_1_3_'></a>[Step 3) Exact method: Simulate $V_T$ with CDF by Laplace transform](#toc0_)
Use Laplace transform to get the CDF of $\int_t^u \frac{ds}{X_s}$, then get the price.

##### <a id='toc1_1_7_1_4_'></a>[Step 3) Almost exact method: Simulate $V_T$ with assumed distribution](#toc0_)
Then we use the characteristic function to generate moment $M1$, $M2$. For simplicity, we assume that $\int_t^u \frac{ds}{X_s}$ follows the Inverse-Gaussian distribution / Gamma distribution / Log-normal distribution, then we can simulate $\int_t^u \frac{ds}{X_s}$.

##### <a id='toc1_1_7_1_5_'></a>[Finally get the option price](#toc0_)
$$
log(S_u) \sim N(log(S_t)+r(u-t)-\frac{1}{2}\int_t^u \frac{ds}{X_s}+\rho\int_t^u({\sqrt{X_s})^{-1}dW_s^1},  \sigma^2(t,u))\tag{8}
$$
where 
$$
\sigma^2(t, u) = (1-\rho^2)\int_t^u \frac{ds}{X_s}
$$
The option price is $C_{BS}(K,S_u, \sigma(t, u))$

In [218]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [219]:
import numpy as np
import time
import scipy.stats as spst
import scipy.special as spsp
import copy 
#import mp math as mp

import sys
sys.path.insert(sys.path.index("")+1, "C:/Users/27261/Desktop/3_Courses in PHBS/3_09_AppliedStochasticProcess/Project_sv32_EMC")
import pyfeng as pf
import pyfeng.ex as pfex
np.set_printoptions(precision=4)

Run only one below

## <a id='toc1_2_'></a>[Case I: Eq. (4.2) in Baldeaux (2012)](#toc0_)

In [524]:
mr, theta, vov, rho, sigma, intr = 2.0, 1.5, 0.2, -0.5, 1.0, 0.05
strike, spot, texp = 1, 1, 1
p_exact = 0.443059

## <a id='toc1_3_'></a>[Case II: Set 2 in Kouarfate et al. (2021)](#toc0_)

In [593]:
sigma, mr, theta, vov, rho = 0.06, 22.84, 0.218, 8.56, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.5
p_exact = np.array([10.364, 7.386, 4.938])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([27.6964, 26.2202, 24.8061])

## <a id='toc1_4_'></a>[Case III:in Kouarfate et al. (2021)](#toc0_)

In [614]:
sigma, mr, theta, vov, rho = 0.06, 20.48, 0.218, 3.20, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.5
p_exact = np.array([11.724, 8.999, 6.710])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([32.8172, 31.9686, 31.1591])

## <a id='toc1_5_'></a>[Case IV:](#toc0_)
Near expiration and atm

In [654]:
# assumption r: discount rate  0 

vov = 0.3
mr = 0.7
spot = 49 # asset price
texp= 0.1
intr = 0
theta = 0.6

strike = np.array([47, 48, 49, 50, 51]) # strike price 

sigma = 1 # sigma
rho= 0.5
p_exact = np.array([7.04, 6.5, 6.121,  5.7006, 5.305])

## Case V:
Near expiration only

In [736]:
vov = 0.3
mr = 0.7
spot = 50 # asset price
texp= 0.001
intr = 0
theta = 0.6

strike = np.array([10, 20, 40]) # strike price 

sigma = 1 # sigma
rho= 0.5
p_exact = np.array([40, 30, 10])

## Case VI:
atm only

In [755]:
vov = 3.3
mr = 0.7
spot = 49 # asset price
texp= 1
intr = 0
theta = 0.6

strike = np.array([47, 48, 49, 50, 51]) # strike price 

sigma = 1 # sigma
rho= 0.5
p_exact = np.array([12.6802, 12.333, 11.999, 11.678, 11.37])

## <a id='toc1_6_'></a>[Pricing with Time Discretization using Euler/Milstein scheme, Exact Stepping, Almost Exact Stepping](#toc0_)

In [748]:
# Euler/Milstein scheme, here dt should be small enough (dt=1/500 for Case I, III, IV, V; dt=1/5000 for Case II;dt=1/50000 for Case VI)
# If vov is high, dt should be large as 1/500, Case II is an example
m = pfex.Sv32McTimeStep(sigma, vov, rho, mr, theta, intr)
m.set_num_params(n_path=1.6e5, dt=1/50000, rn_seed=123456)
m.scheme = 1  
m.correct_fwd = False

In [749]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

[12.6905 12.343  12.0092 11.6884 11.3802]
CPU times: total: 14min 7s
Wall time: 14min 16s


In [756]:
# Exact Stepping with 1 / NCX2
m.set_num_params(n_path=1.6e5, dt=1/500, rn_seed=123456)
m.scheme = 2 

In [757]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

[ 4.9865e-05 -2.0264e-04  2.0030e-05  3.2540e-04  1.4112e-04]
CPU times: total: 9.11 s
Wall time: 9.14 s


In [758]:
# Almost Exact Stepping with Poison-Gamma distribution
m.set_num_params(n_path=1.6e5, dt=1/500, rn_seed=123456)
m.scheme = 3

In [759]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

[0.0722 0.0714 0.0711 0.071  0.0703]
CPU times: total: 16.3 s
Wall time: 16.5 s


## <a id='toc1_7_'></a>[Pricing with Exact Simulation](#toc0_)
## Exact simulation **cannot** price the atm option or near expiration

In [760]:
m1 = pfex.Sv32McBaldeaux2012Exact(sigma, vov, rho, mr, theta, intr)
m1.set_num_params(n_path=10000, rn_seed=123456, dt=None)
m1.correct_fwd = False

In [761]:
%%time
bias = m1.price(strike, spot, texp) - p_exact
print(bias)
# cannot price the atm option or near expiration

[nan nan nan nan nan]
CPU times: total: 3.23 s
Wall time: 3.23 s


c:\Users\27261\anaconda3\Lib\site-packages\scipy\optimize\_zeros_py.py:482: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)
d:\GitHUb\Project_sv32_EMC\pyfeng\sv32_mc2.py:148: RuntimeWarning: invalid value encountered in sqrt
  sigma_cond = np.sqrt(


## <a id='toc1_8_'></a>[Pricing with IG approximation (Almost Exact Simulation)](#toc0_)
## Almost exact simulation **cannot** price the atm option (Case VI as an example) and options far from expiration date (Case II and Case III as examples, though in Case III seems good **coincidently**, which is because vov is small in Case III.)

In [762]:
# Almost exact simulation cannot price the atm option  and options far from expiration date
m2 = pfex.Sv32McChoiKwok2023Ig(sigma, vov, rho, mr, theta, intr)
m2.set_num_params(n_path=100000, rn_seed=123456, dt=None)
m2.correct_fwd = False

In [763]:
%%time
bias = m2.price(strike, spot, texp) - p_exact
print(bias)

[6.7542e+43 6.7542e+43 6.7542e+43 6.7542e+43 6.7542e+43]
CPU times: total: 469 ms
Wall time: 478 ms


# <a id='toc2_'></a>[Pricing with FFT method](#toc0_)
## FFT method **cannot** price options near expiration

In [764]:
m3 = pf.sv_fft.Sv32Fft(sigma, vov, rho, mr, theta, intr)
m3.correct_fwd = False

In [765]:
%%time
bias = m3.price(strike, spot, texp) - p_exact
print(bias)

[0.0754 0.0751 0.0753 0.0755 0.0752]
CPU times: total: 109 ms
Wall time: 111 ms


# <a id='toc3_'></a>[Pricing with approximation IV](#toc0_)
## Approximation IV **can only** price atm options near expiration 

3/2 model
$$ d S_t = S_t dt + dW_t $$
$$d v_t=\kappa v_t\left(\theta-v_t\right) d t+v v_t^{3 / 2} d Z_t $$
$$v_t = \sigma_t^2 $$
$$ d \sigma_t=d \sqrt{v_t}=\frac{1}{2} \frac{d v_t}{\sqrt{v_t}}-\frac{1}{8} \frac{\left(d v_t\right)^2}{v_t \sqrt{v_t}} $$
$$ d \sigma_t = (v\sigma_t^2/2-v^2\sigma_t^3/8)dZ_t -\kappa(\sigma_t^3-\sigma_t\theta)/2d_t  $$

model setting

$$ dS_t = S_tdt + \sigma_tS_tdW_t^{(1)} $$

$$ d\sigma_t = a(\sigma_t)dt+b(\sigma_t)[\rho dW_1^{(1)}+\sqrt{1-\rho^2}dW_t^{(2)}] $$
$$ a(\sigma_t) = (v\sigma_t^2/2-v^2\sigma_t^3/8) $$
$$ a'(\sigma_t) = v\sigma_t^2-v^2\sigma_t^2*3/8 $$
$$ b(\sigma_t) = -\kappa(\sigma_t^3-\sigma_t\theta)/2 $$
$$ b'(\sigma_t)= -\kappa(3 \sigma_t^2-\theta)/2 $$

$$
\begin{aligned}
&I_1(\theta_{IV} ; \sigma)=-\frac{\rho b \theta_{IV}}{2},\\
&\begin{aligned}
I_2(\theta_{IV} ; \sigma)= & \left(-\frac{5}{12} \frac{\rho^2 b^2}{\sigma}+\frac{1}{6} \frac{b^2}{\sigma}+\frac{1}{6} \rho^2 b b^{\prime}\right) \theta_{IV}^2 \\
& +\frac{a}{2}+\frac{\rho b \sigma}{4}+\frac{1}{24} \frac{\rho^2 b^2}{\sigma}+\frac{1}{12} \frac{b^2}{\sigma}-\frac{1}{6} \rho^2 b b^{\prime}
\end{aligned}
\end{aligned}
$$

$$
I(\theta_{IV}, \tau ; \sigma)=\sigma+I_1(\theta_{IV} ; \sigma) \sqrt{\tau}+I_2(\theta_{IV} ; \sigma) \tau+O(\tau \sqrt{\tau})
$$

In [766]:
X_t = np.log(spot/strike) #option moneyness
theta_IV= X_t/(sigma*np.sqrt(texp)) # moneyness degree
a = vov*sigma**2/2+vov**2*sigma**3/8
b= -mr*(sigma**3-sigma*theta)/2
b_prime = -mr*(3*sigma**2-theta)/2

In [767]:
I1 = - theta_IV*b*rho/2
I2 =(-5/12*rho**2*b**2/sigma+1/6*b**2/sigma+1/6*rho**2*b*b_prime)*theta_IV**2+ a/2+rho*b*sigma/4+1/24*rho**2*b**2/sigma+1/12*b**2/sigma-1/6*rho**2*b*b_prime
I = sigma + I1*np.sqrt(texp)+I2*texp

In [768]:
m4 = pf.Bsm(sigma=I, intr=0.0, divr=0.0)
price = m4.price(strike=strike, spot=spot, texp=texp)
bias = price - p_exact
print(bias)

[26.0619 26.2933 26.513  26.7213 26.9181]
